<a href="https://colab.research.google.com/github/ovbystrova/Interference/blob/master/Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Author Verification Using Common N-Gram Profiles of Text Documents](https://www.aclweb.org/anthology/C14-1038.pdf)

In [0]:
import numpy as np
from collections import Counter
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

## Distances

### Implement the distance formula from the article:
$$D(P_1,P_2)=\sum_{x\in(P_1 \cup P_2)}\left(\frac{fP_1(x)-fP_2(x)}{\frac{1}{2} (fP_1(x)+fP_2(x))}\right) ^2$$

where $P_1$ and $P_2$ are n-gram profiles of 2 documents,  and $fP_i(x)$ - length-normilized frequency of an n-gram $x$ in a profile $P_i$

In [0]:
def distance(profile1, profile2):
  """
  calculates distance between 2 document profiles

  :param profile1: collections.Counter, token frequencies in profile 1
  :param profile2: collections.Counter, token frequencies in profile 2
  :return: float, distance
  """
  vocab = set(profile1.keys()).union(set(profile2.keys()))
  diffs = []
  for word in vocab:
    fp1 = profile1[word]
    fp2 = profile2[word]
    diff = ((fp1 - fp2)/((fp1 + fp2)/2)) ** 2
    diffs.append(diff)
  return np.sum(diffs)

### Implement the radius distance form the article:
$$r(d_i, u, A) = \frac{D(d_i, u)}{D^{max}(d_i, A)}$$
where $D^{max}(d_i, A)$ is $max_{j=1}^{m=|A|}(d_i, A_j)$, or the maximal distance between a known-author document $d_i$ and all other documents from known-author collection $A$

In [0]:
def radius(di, u, A):
  """
  compares proximity between an unknown document and a known one
  :param di: collections.Counter, token frequencies in a known document i
  :param u: collections.Counter, token frequencies in an unknown document
  :param A: list of collections.Counter, token frequencies of all know documents
  :return: float, distance
  """
  return(distance(di, u)/np.max([distance(di, ai) for ai in A]))

In [0]:
def radius_distance(u, A):
  """
  compares proximity between an unknown document and all known ones
  :param u: collections.Counter, token frequencies in an unknown document
  :param A: list of collections.Counter, token frequencies of all know documents
  :return: float, distance
  """
  dists = []
  for di in A:
    dists.append(radius(di, u, A))
  return np.mean(dists)

## Compile a singe classifier

Single classifier truncates given n-gram frequency profiles to the length specified, and then uses radius distance to predict the class of each example. It also provides distance scores for each class for each example.

In [0]:
class SingleClassifier():
    """
    classifies texts as belonging to several classes given true classes

    :param profiles: list of collection.Counter, n-gram profiles
    :param y_true: list of str or int, true classes of each text from profiles
    :param p_length: int, length of truncated profile
    :param classes: list of str or int, possible classes 

    Attributes:
      y_true, p_length, profiles (already truncated to p_length), classes
    """
    def __init__(self, profiles, y_true, p_length, classes):
        self.y_true = y_true
        self.p_length = p_length  
        self.profiles = self.truncate(profiles, p_length)
        self.classes = classes
    
    @staticmethod
    def truncate(profiles, p_length):
        """
        truncates a profile to a given length specified by p_length

        :param profiles: list of collection.Counter, n-gram profiles
        :param p_length: int, length of truncated profile, 
                         if more than actual profile length, remains unchanged
        :return: list of collection.Counter, text n-gram profiles truncated to
                 the length of p_length
        """
        profiles = [profile.most_common(p_length) for profile in profiles]
        profiles = [make_counter(profile) for profile in profiles]
        return profiles

    def forward_one(self, x):  
        """
        predicts the class of a given n-gram profile x

        :param x: collection.Counter, n-gram profile to predict the class of
        :return y: str or int, predicted class
        :return class_dist: dict of {str or int: float}, proximity to each class
        """
        class_dist = {}
        for c in self.classes:
            y_ids = np.where(np.array(self.y_true)==c)  # Выбрать все мемберы этого класса кроме х
            y_c = np.array(self.profiles)[y_ids]
            
            if x in y_c:
                y_c = np.delete(y_c, np.argwhere(y_c==x))
            distance = radius_distance(x, y_c)  
            class_dist[c] = distance
        
        return min(class_dist, key=class_dist.get), class_dist

    def forward_all(self):
        """
        predicts the class of each profile form profiles

        :return: list of tuples (y, class_dict)
        :type y: str or int, predicted class
        :type class_dist: dict of {str or int: float}, proximity to each class
        """
        return [self.forward_one(x) for x in self.profiles]

    @staticmethod
    def only_classes(response):
      """
      retrieves only class predictions from forward_all()

      :param response: list of tuples (y, class_dict)
        :type y: str or int, predicted class
        :type class_dist: dict of {str or int: float}, proximity to each class
      :return: list of str or int, classes
      """
      return [el[0] for el in response]
    
    @staticmethod
    def only_distances(response, arr=False):
      """
      retrieves only distance predictions from forward_all()

      :param response: list of tuples (y, class_dict)
        :type y: str or int, predicted class
        :type class_dist: dict of {str or int: float}, proximity to each class
      :param arr: bool, if to return list of lists, optional, default False
      :return: 
        if arr == False:
          list of dict of {str or int: float}, proximity to each class 
        if arr == True:
          :return dists: list of lists, proximity to each class
          :return classes: list of int or str, classes 
                           in the same order as in dists
      """
      if not arr:
        return [el[1] for el in response]
      else:
        classes = [el[0] for el in sorted(response[0][1].items())]
        dists = [[el[1] for el in sorted(x[1].items())] for x in response]
        return dists, classes

    
    #TODO reshape only_distances to [class[batch]] (this is transpose, but that is expensive)

### Test the single classifier

#### Preprocessing for simulated data

In [0]:
def preprocess(text):
  return(Counter(text.split()))

def make_counter(commons):
    return Counter({x[0]: x[1] for x in commons})

#### Simulate data

In [0]:
texts = ['I am so sad I am so tired', 'I am so useless and crying and tired so tired so tired', 
         'I am so sad so sad so sad', 'I am so sad and so tired',
         
         'Wow a velocaraptor', 'She is no longer a veloceraptor',
         'I wish we could be a velocaraptor', 'velocaraptor veloceraptor veloceraptor']

ys = ['sad', 'sad', 'sad', 'sad', 'dino', 'dino', 'dino', 'dino']

profs = [preprocess(text) for text in texts]

#### Test

In [0]:
classes = ['sad', 'dino']

In [0]:
cl = SingleClassifier(profs, ys, 10, classes=classes)
cl.forward_one(preprocess(texts[0]))

('sad', {'dino': 1.0155122655122655, 'sad': 0.5214161917552966})

In [0]:
cl.forward_all()

[('sad', {'dino': 1.0155122655122655, 'sad': 0.5214161917552966}),
 ('dino', {'dino': 1.2467532467532467, 'sad': 2.2174809624122838}),
 ('sad', {'dino': 0.8961038961038962, 'sad': 0.7055166523861863}),
 ('sad', {'dino': 1.1298701298701297, 'sad': 0.4696971554169289}),
 ('dino', {'dino': 0.5367965367965367, 'sad': 1.9547890946953117}),
 ('dino', {'dino': 1.2037037037037037, 'sad': 2.6415204838078177}),
 ('dino', {'dino': 1.1913419913419914, 'sad': 2.418766139952783}),
 ('dino', {'dino': 0.5401635401635402, 'sad': 1.7258786316578099})]

#### Compute performance

In [0]:
y_pred = cl.only_classes(cl.forward_all())
y_pred

['sad', 'dino', 'sad', 'sad', 'dino', 'dino', 'dino', 'dino']

In [0]:
cl.y_true

['sad', 'sad', 'sad', 'sad', 'dino', 'dino', 'dino', 'dino']

accuracy

In [0]:
accuracy_score(cl.y_true, y_pred)

0.875

precision

In [0]:
precision_score(cl.y_true, y_pred, pos_label='dino')

0.8

In [0]:
precision_score(cl.y_true, y_pred, pos_label='sad')

1.0

recall

In [0]:
recall_score(cl.y_true, y_pred, pos_label='dino')

1.0

In [0]:
recall_score(cl.y_true, y_pred, pos_label='sad')

0.75

$f_1$score

In [0]:
f1_score(cl.y_true, y_pred, pos_label='sad')

0.8571428571428571

In [0]:
f1_score(cl.y_true, y_pred, pos_label='dino')

0.888888888888889

ROC AUC score

In [0]:
def softmax(scores, inverse=True):
  """
  normilizes class scores, making them correspond to probability, adding up to 1

  :param scores: list of int or float, scores
  :inverse: bool, if lower scores indicate higher probability,
            optional, default True
  :return: list of int or float, normilized probability-like scores
  """
  if inverse:
    scores = [-1*score for score in scores]
  normalizing_constant = np.sum(scores)
  normalized_scores = [score/normalizing_constant for score in scores]
  return normalized_scores

In [0]:
y_score = cl.only_distances(cl.forward_all(), arr=True)[0]  # get scores from dict
y_score = [softmax(scores) for scores in y_score]  # normilize scores
y_score

[[0.6607414032255609, 0.3392585967744391],
 [0.359892885837406, 0.640107114162594],
 [0.5594982512859819, 0.4405017487140182],
 [0.7063598638599082, 0.29364013614009177],
 [0.2154437439403306, 0.7845562560596694],
 [0.3130386279200781, 0.6869613720799219],
 [0.3300017473201595, 0.6699982526798405],
 [0.23837311894746394, 0.7616268810525361]]

In [0]:
cl.y_true

['sad', 'sad', 'sad', 'sad', 'dino', 'dino', 'dino', 'dino']

In [0]:
y_true = [[1, 0] if label == 'sad' else [0, 1] for label in cl.y_true]
y_true

[[1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [0, 1]]

In [0]:
roc_auc_score(y_true, y_score, multi_class='ovo')

1.0

In [0]:
roc_auc_score(y_true, y_score, multi_class='ovr')

1.0

### **TODO: Test on real data**

## Comile an ensemble classifier 

Ensamble classifier consists of multiple single calssifiers. They differ in n-gram profile type (characters or words) and profile truncation length. Each combination is present.

Options from the article:
- size of N-grams (n)
    - from 3 to 10 for characters
    - from 1 to 3 for words
- size of a profile Number of n-grams (L) 200, 500, 1000, 1500, 2000, 2500, 3000. 

The prediction is either majority voting or weighted votion on the distance sum.

In [0]:
class EnsambleClassifier():
    """
    classifies texts as belonging to several classes given true classes
    by combining several single classifiers

    :param profiles_multiple: list of list of collection.Counter, 
                              n-gram profiles (of each n-gram type)
    :param y_true: list of str or int, true classes of each text from profiles
    :param p_length_options: itearble of int, options for length of 
                             truncated profiles
    :param classes: list of int or str, possible classes

    Attributes:
      y_true, p_length_options, profiles_multiple, classes,
      classifiers: list of SingleClassifier, each parameter combination
    """
    def __init__(self, profiles_multiple, y_true, p_length_options, classes):
        self.y_true = y_true
        self.p_length_options = p_length_options  
        self.profiles_multiple = profiles_multiple
        self.classes = classes
        self.classifiers = self.create_ensamble()
        
    
    def create_ensamble(self):
      """
      creates a SingleClassifier for each parameter combination

      :return: list of SingleClassifier, each parameter combination
      """
      classifiers = []
      for profiles in self.profiles_multiple:
        for p_length in self.p_length_options:
          classifiers.append(SingleClassifier(profiles, self.y_true, p_length, self.classes))
      return classifiers
    

    @staticmethod
    def majority_vote(votes):
      """
      majority voting, 
        if several classes have equal number of votes 
        returns the alphabetically first class

      :param votes: list of lsits str or int, 
        predictions of each classifier on each object
      :return: list int or str, resulting prediction
      """
      votes = np.array(votes).T
      pred = [Counter(vote).most_common(1)[0][0] for vote in votes]
      return pred


    def confidence_summing(self, classifier_distances):
      """
      confidence voting, distances to classes are summed

      :param classifier_distances: list of list of dicts 
                        {str or int: float or int}, 
                        confidences of each classifier on each object
      :return: list of dict {str or int: float or int}, 
              resulting confidences on each object
      """
      cummulative_distances = classifier_distances[0]
      if len(classifier_distances) == 1:
        return cummulative_distances
      else:
        for classifier in classifier_distances[1:]:
          for i, element in enumerate(classifier):
            for c in self.classes:
              cummulative_distances[i][c] += element[c]
        return cummulative_distances


    def weighted_vote(self, classifier_distances):
      """
      wheighted voting, 
        distances to classes are summed for each object, 
        minimal distance is the final predicted class

      :param classifier_distances: list of list of dict {str or int: float or int}, 
                        confidences of each classifier
      :return: list of int or str, resulting prediction on each object
      """
      sum_distances = self.confidence_summing(classifier_distances)
      return [min(class_dist, key=class_dist.get) for class_dist in sum_distances]
      

    def forward_ensamble(self, method='majority', confidence=False):
      """
      predicts classes using the ensemble

      :param method: str ('majority' or 'weight'), nethod of voting to use,
                     optional, default 'majority'
      :param confidence: bool, whether to return proximity score for each class,
                         optional, default False
      :return: 
        if confidence == False:
          list of int or str, resulting prediction on each object
        if confidence == true:
          list of tuple (res, dist)
          :return res: int or str, resulting prediction
          :return dists: dict {str or int: float or int}, proximity to each class
      """
      responses = [classifier.forward_all() for classifier in self.classifiers]
      dists = False
      if method == 'majority':
        votes = [SingleClassifier.only_classes(response) for response in responses]
        res = self.majority_vote(votes)
      elif method == 'weight':
        dists = [SingleClassifier.only_distances(response) for response in responses]
        res = self.weighted_vote(dists)
      else:
        raise KeyError(f'Expected method "majority" or "weight", got "{method}"')
      if confidence:
        if dists:
          return list(zip(res, self.confidence_summing(dists)))
        else:
          dists = [SingleClassifier.only_distances(response) for response in responses]
          return list(zip(res, self.confidence_summing(dists)))
      else:
        return res

### Test the ensemble classifier

In [0]:
classes = ['sad', 'dino']

#### single classifiers

In [0]:
ec = EnsambleClassifier([profs], ys, range(1, 8), classes=classes)
classifiers = ec.classifiers
classifiers[0].forward_all()

[('dino', {'dino': 0.7638888888888888, 'sad': 49.99999999999999}),
 ('sad', {'dino': 1.0, 'sad': 0.51}),
 ('sad', {'dino': 1.0, 'sad': 0.51}),
 ('sad', {'dino': 1.0, 'sad': 0.3466666666666667}),
 ('sad', {'dino': 1.0, 'sad': 1.0}),
 ('sad', {'dino': 1.0, 'sad': 1.0}),
 ('sad', {'dino': 1.0, 'sad': 0.7638888888888888}),
 ('sad', {'dino': 1.0, 'sad': 1.0})]

#### majority voting

In [0]:
classifier_votes = [classifier.only_classes(classifier.forward_all()) for classifier in classifiers]
classifier_votes

[['dino', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad'],
 ['dino', 'sad', 'sad', 'sad', 'dino', 'dino', 'sad', 'dino'],
 ['sad', 'dino', 'sad', 'sad', 'dino', 'dino', 'dino', 'dino'],
 ['sad', 'dino', 'sad', 'sad', 'dino', 'dino', 'dino', 'dino'],
 ['sad', 'dino', 'sad', 'sad', 'dino', 'dino', 'dino', 'dino'],
 ['sad', 'dino', 'sad', 'sad', 'dino', 'dino', 'dino', 'dino'],
 ['sad', 'dino', 'sad', 'sad', 'dino', 'dino', 'dino', 'dino']]

In [0]:
ec.majority_vote(classifier_votes)

['sad', 'dino', 'sad', 'sad', 'dino', 'dino', 'dino', 'dino']

In [0]:
ec.forward_ensamble(method='majority')

['sad', 'dino', 'sad', 'sad', 'dino', 'dino', 'dino', 'dino']

In [0]:
ec.forward_ensamble()

['sad', 'dino', 'sad', 'sad', 'dino', 'dino', 'dino', 'dino']

In [0]:
ec.forward_ensamble(confidence=True)

[('sad', {'dino': 6.483633958633959, 'sad': 54.48789890641409}),
 ('dino', {'dino': 7.255681818181818, 'sad': 18.470386037161486}),
 ('sad', {'dino': 6.470806277056277, 'sad': 4.732971734846115}),
 ('sad', {'dino': 6.9304653679653665, 'sad': 3.404503882927594}),
 ('dino', {'dino': 5.35465367965368, 'sad': 12.833764900333788}),
 ('dino', {'dino': 7.255952380952381, 'sad': 15.423582878916044}),
 ('dino', {'dino': 7.738816738816738, 'sad': 12.422209304651492}),
 ('dino', {'dino': 5.110137085137086, 'sad': 11.432901379745811})]

#### weighted voting

In [0]:
classifier_dists = [classifier.only_distances(classifier.forward_all()) for classifier in classifiers]

In [0]:
ec.confidence_summing(classifier_dists)

[{'dino': 6.483633958633959, 'sad': 54.48789890641409},
 {'dino': 7.255681818181818, 'sad': 18.470386037161486},
 {'dino': 6.470806277056277, 'sad': 4.732971734846115},
 {'dino': 6.9304653679653665, 'sad': 3.404503882927594},
 {'dino': 5.35465367965368, 'sad': 12.833764900333788},
 {'dino': 7.255952380952381, 'sad': 15.423582878916044},
 {'dino': 7.738816738816738, 'sad': 12.422209304651492},
 {'dino': 5.110137085137086, 'sad': 11.432901379745811}]

In [0]:
ec.weighted_vote(classifier_dists)

['dino', 'dino', 'sad', 'sad', 'dino', 'dino', 'dino', 'dino']

In [0]:
ec.forward_ensamble(method='weight')

['dino', 'dino', 'sad', 'sad', 'dino', 'dino', 'dino', 'dino']

In [0]:
ec.forward_ensamble(method='weight', confidence=True)

[('dino', {'dino': 12.203379028379027, 'sad': 58.97579781282818}),
 ('dino', {'dino': 13.511363636363637, 'sad': 36.43077207432297}),
 ('sad', {'dino': 11.941612554112554, 'sad': 8.95594346969223}),
 ('sad', {'dino': 12.860930735930733, 'sad': 6.462341099188523}),
 ('dino', {'dino': 9.70930735930736, 'sad': 24.667529800667577}),
 ('dino', {'dino': 13.511904761904763, 'sad': 29.847165757832084}),
 ('dino', {'dino': 14.477633477633475, 'sad': 24.080529720414095}),
 ('dino', {'dino': 9.220274170274172, 'sad': 21.865802759491622})]

#### incorrect method

In [0]:
ec.forward_ensamble(method='aaa')

KeyError: ignored